<h3> Case study </h3>

먼저 아래 사이트의 데이터를 가져오자. <br>
시간과 데이터 종류가 정리된 통화량 데이터이다.

* https://www.shanelynn.ie/wp-content/uploads/2015/06/phone_data.csv

In [1]:
!wget https://www.shanelynn.ie/wp-content/uploads/2015/06/phone_data.csv

--2021-11-12 12:41:28--  https://www.shanelynn.ie/wp-content/uploads/2015/06/phone_data.csv
Resolving www.shanelynn.ie (www.shanelynn.ie)... 104.236.88.249
Connecting to www.shanelynn.ie (www.shanelynn.ie)|104.236.88.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40576 (40K) [text/csv]
Saving to: ‘phone_data.csv’

phone_data.csv      100%[===================>]  39.62K  --.-KB/s    in 0.007s  

2021-11-12 12:41:28 (5.21 MB/s) - ‘phone_data.csv’ saved [40576/40576]



In [17]:
import pandas as pd
df_phone = pd.read_csv("phone_data.csv")
df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,15/10/14 06:58,34.429,data,2014-11,data,data
1,1,15/10/14 06:58,13.000,call,2014-11,Vodafone,mobile
2,2,15/10/14 14:46,23.000,call,2014-11,Meteor,mobile
3,3,15/10/14 14:48,4.000,call,2014-11,Tesco,mobile
4,4,15/10/14 17:27,4.000,call,2014-11,Tesco,mobile


In [18]:
# 날짜 데이터가 string 형태로 저장됨.
df_phone.dtypes

index             int64
date             object
duration        float64
item             object
month            object
network          object
network_type     object
dtype: object

위처럼 날짜 데이터가 스트링 타입으로 저장됐을 때 아래와 같은 코드를 통해 날짜 데이터로 바꿔줄 수 있다.

* **DateFrame 객체[바꿀 열].apply(dateutil.parser.parse, dayfirst = True)**

In [19]:
import dateutil
# dateutil.parser.parse 문자형 데이터를 날짜형 데이터로 변형시켜주는 함수다.
df_phone["date"] = df_phone["date"].apply(dateutil.parser.parse, dayfirst=True)
df_phone.dtypes

index                    int64
date            datetime64[ns]
duration               float64
item                    object
month                   object
network                 object
network_type            object
dtype: object

In [20]:
df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile


아래처럼 **특정 열에만 함수를 적용**할 수 있다.

In [21]:
df_phone.groupby('month', as_index = False).agg({'duration': 'sum'})

,month,duration
0,2014-11,26639.441
1,2014-12,14641.870
2,2015-01,18223.299
3,2015-02,15522.299
4,2015-03,22750.441


**아래처럼 각 열에 다른 함수를 적용할 수 있다.**

In [22]:
df_phone.groupby(['month', 'item']).agg({'duration': sum, 'network_type': 'count', 'date': 'first'})

duration  network_type                date
month   item                                             
2014-11 call  25547.000           107 2014-10-15 06:58:00
        data    998.441            29 2014-10-15 06:58:00
        sms      94.000            94 2014-10-16 22:18:00
2014-12 call  13561.000            79 2014-11-14 17:24:00
        data   1032.870            30 2014-11-13 06:58:00
        sms      48.000            48 2014-11-14 17:28:00
2015-01 call  17070.000            88 2014-12-15 20:03:00
        data   1067.299            31 2014-12-13 06:58:00
        sms      86.000            86 2014-12-15 19:56:00
2015-02 call  14416.000            67 2015-01-15 10:36:00
        data   1067.299            31 2015-01-13 06:58:00
        sms      39.000            39 2015-01-15 12:23:00
2015-03 call  21727.000            47 2015-02-12 20:15:00
        data    998.441            29 2015-02-13 06:58:00
        sms      25.000            25 2015-02-19 18:46:00

각 열에 다른 함수를 적용하면서 **한 열에 여러 함수를 적용**할 수도 있다.

In [23]:
df_phone.groupby(['month', 'item']).agg({'duration': [min, max, sum], 
                                         'network_type': 'count', 
                                         'date': [min, 'first', 'nunique']})

duration             ...                date        
                  min        max  ...               first nunique
month   item                      ...                            
2014-11 call    1.000   1940.000  ... 2014-10-15 06:58:00     104
        data   34.429     34.429  ... 2014-10-15 06:58:00      29
        sms     1.000      1.000  ... 2014-10-16 22:18:00      79
2014-12 call    2.000   2120.000  ... 2014-11-14 17:24:00      76
        data   34.429     34.429  ... 2014-11-13 06:58:00      30
        sms     1.000      1.000  ... 2014-11-14 17:28:00      41
2015-01 call    2.000   1859.000  ... 2014-12-15 20:03:00      84
        data   34.429     34.429  ... 2014-12-13 06:58:00      31
        sms     1.000      1.000  ... 2014-12-15 19:56:00      58
2015-02 call    1.000   1863.000  ... 2015-01-15 10:36:00      67
        data   34.429     34.429  ... 2015-01-13 06:58:00      31
        sms     1.000      1.000  ... 2015-01-15 12:23:00      27
2015-03 call    2.000  10528.000  ... 2015-02-12 20:15:00      47
        data   34.429     34.429  ... 2015-02-13 06:58:00      29
        sms     1.000      1.000  ... 2015-02-19 18:46:00      17

[15 rows x 7 columns]

<h3> Pivot table / Crosstab </h3>

<h4> Pivot table </h4>

* **DataFrame 객체.pivot_table( <br>
values = [값으로 설정할 열들],<br>
index = [행으로 설정할 열들],<br>
columns = [열로 설정할 열들],<br>
aggfunc = [적용할 함수들])**

먼저 데이터를 먼저 가져와 보자.

In [24]:
df_phone = pd.read_csv('phone_data.csv')
df_phone['date'] = df_phone['date'].apply(dateutil.parser.parse, dayfirst = True)
df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile


**pivot_table() 함수**를 사용해보자.

In [26]:
df_phone.pivot_table(values = ['duration'],
                     index = [df_phone.month, df_phone.item],
                     columns = df_phone.network,
                     aggfunc = 'sum',
                     fill_value = 0)

duration                        ...                                 
network        Meteor Tesco  Three Vodafone  ... landline special voicemail world
month   item                                 ...                                 
2014-11 call     1521  4045  12458     4316  ...     2906       0       301     0
        data        0     0      0        0  ...        0       0         0     0
        sms        10     3     25       55  ...        0       1         0     0
2014-12 call     2010  1819   6316     1302  ...     1424       0       690     0
        data        0     0      0        0  ...        0       0         0     0
        sms        12     1     13       18  ...        0       0         0     4
2015-01 call     2207  2904   6445     3626  ...     1603       0       285     0
        data        0     0      0        0  ...        0       0         0     0
        sms        10     3     33       40  ...        0       0         0     0
2015-02 call     1188  4087   6279     1864  ...      730       0       268     0
        data        0     0      0        0  ...        0       0         0     0
        sms         1     2     11       23  ...        0       2         0     0
2015-03 call      274   973   4966     3513  ...    11770       0       231     0
        data        0     0      0        0  ...        0       0         0     0
        sms         0     4      5       13  ...        0       0         0     3

[15 rows x 9 columns]

아래처럼 **groupby() 함수**를 사용해 피벗 테이블을 만들 수 있다. <br>
pivot_table() 함수가 더 직관적이다.

In [27]:
df_phone.groupby(['month', 'item', 'network'])['duration'].sum().unstack()

network       Meteor   Tesco    Three  ...  special  voicemail  world
month   item                           ...                           
2014-11 call  1521.0  4045.0  12458.0  ...      NaN      301.0    NaN
        data     NaN     NaN      NaN  ...      NaN        NaN    NaN
        sms     10.0     3.0     25.0  ...      1.0        NaN    NaN
2014-12 call  2010.0  1819.0   6316.0  ...      NaN      690.0    NaN
        data     NaN     NaN      NaN  ...      NaN        NaN    NaN
        sms     12.0     1.0     13.0  ...      NaN        NaN    4.0
2015-01 call  2207.0  2904.0   6445.0  ...      NaN      285.0    NaN
        data     NaN     NaN      NaN  ...      NaN        NaN    NaN
        sms     10.0     3.0     33.0  ...      NaN        NaN    NaN
2015-02 call  1188.0  4087.0   6279.0  ...      NaN      268.0    NaN
        data     NaN     NaN      NaN  ...      NaN        NaN    NaN
        sms      1.0     2.0     11.0  ...      2.0        NaN    NaN
2015-03 call   274.0   973.0   4966.0  ...      NaN      231.0    NaN
        data     NaN     NaN      NaN  ...      NaN        NaN    NaN
        sms      NaN     4.0      5.0  ...      NaN        NaN    3.0

[15 rows x 9 columns]

<h3> Merge / Concat </h3>

<h4> Merge </h4>

**merge() 함수**를 이용해 두 개의 데이터를 합칠 수 있다.

* **pd.merge(DataFrame 객체1, DataFrame 객체2, on = '기준열')**



In [31]:
raw_data = {
    "subject_id": ["1", "2", "3", "4", "5", "7", "8", "9", "10", "11"],
    "test_score": [51, 15, 15, 61, 16, 14, 15, 1, 61,  16]
}
df_a = pd.DataFrame(raw_data, columns = ['subject_id', 'test_score'])
df_a

,subject_id,test_score
0,1,51
1,2,15
2,3,15
3,4,61
4,5,16
5,7,14
6,8,15
7,9,1
8,10,61
9,11,16


In [32]:
raw_data = {
    "subject_id": ["4", "5", "6", "7", "8"],
    "first_name": ["Billy", "Brian", "Bran", "Bryce", "Betty"],
    "last_name": ["Bonder", "Black", "Balwner", "Brice", "Btisan"]
}
df_b = pd.DataFrame(raw_data, columns = ["subject_id", "first_name", "last_name"])
df_b

,subject_id,first_name,last_name
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


In [33]:
pd.merge(df_a, df_b, on = "subject_id")

,subject_id,test_score,first_name,last_name
0,4,61,Billy,Bonder
1,5,16,Brian,Black
2,7,14,Bryce,Brice
3,8,15,Betty,Btisan


두 dataframe의 column 이름이 다를 때 아래 방식을 쓸 수 있다.

* **pd.merge(DataFrame 객체1, DataFrame 객체2, left_on = '왼쪽 기준열', right_on = '오른쪽 기준열')** <br>

In [34]:
pd.merge(df_a, df_b, left_on="subject_id", right_on="subject_id")

,subject_id,test_score,first_name,last_name
0,4,61,Billy,Bonder
1,5,16,Brian,Black
2,7,14,Bryce,Brice
3,8,15,Betty,Btisan


merge() 함수를 사용할 때 몇 가지 규칙이 있다.

* **Inner join: 양쪽다 같은 데이터를 가져야 함.**
* **full join: 모두 다 합친다.**
* **left join: 왼쪽에만 있는 것을 합친다.** 
* **right join: 오른쪽에만 있는 것을 합친다.**

**inner join이 기본값이다.**

In [35]:
# df_b의 subject_id = "6" 인 데이터는 출력되지 않는다.
pd.merge(df_a, df_b, on = "subject_id", how = 'left')

,subject_id,test_score,first_name,last_name
0,1,51,NaN,NaN
1,2,15,NaN,NaN
2,3,15,NaN,NaN
3,4,61,Billy,Bonder
4,5,16,Brian,Black
5,7,14,Bryce,Brice
6,8,15,Betty,Btisan
7,9,1,NaN,NaN
8,10,61,NaN,NaN
9,11,16,NaN,NaN


In [36]:
pd.merge(df_a, df_b, on = "subject_id", how = 'right')

,subject_id,test_score,first_name,last_name
0,4,61.0,Billy,Bonder
1,5,16.0,Brian,Black
2,6,NaN,Bran,Balwner
3,7,14.0,Bryce,Brice
4,8,15.0,Betty,Btisan


In [37]:
pd.merge(df_a, df_b, on = "subject_id", how = "outer")

,subject_id,test_score,first_name,last_name
0,1,51.0,NaN,NaN
1,2,15.0,NaN,NaN
2,3,15.0,NaN,NaN
3,4,61.0,Billy,Bonder
4,5,16.0,Brian,Black
5,7,14.0,Bryce,Brice
6,8,15.0,Betty,Btisan
7,9,1.0,NaN,NaN
8,10,61.0,NaN,NaN
9,11,16.0,NaN,NaN


**인덱스 값을 기준**으로 붙이고 싶다면 아래처럼 할 수 있다.

* **pd.merge(DataFrame 객체1, DataFrame 객체2, right_index = True, left_index = True)**

In [38]:
# 양쪽 모두에 있는 값은 _x, _y 가 붙는다.
pd.merge(df_a, df_b, right_index = True, left_index = True)

,subject_id_x,test_score,subject_id_y,first_name,last_name
0,1,51,4,Billy,Bonder
1,2,15,5,Brian,Black
2,3,15,6,Bran,Balwner
3,4,61,7,Bryce,Brice
4,5,16,8,Betty,Btisan


<h4> concat </h4>

**concat() 함수**는 같은 형태의 데이터를 붙이는 작업을 수행한다. <br>
위아래, 양옆 두 가지 방식으로 붙일 수 있다.

* **axis = 0: 위 아래**
* **axis = 1: 양 옆**
* **pd.concat([DataFrame 객체1, DataFrame 객체2])**

In [39]:
raw_data = {
    "subject_id": ["1", "2", "3", "4", "5"],
    "first_name": ["Alex", "Amy", "Allen", "Alice", "Ayoung"],
    "last_name": ["Anderson", "Ackerman", "Ali", "Aoni", "Atiches"]
}
df_a = pd.DataFrame(raw_data, columns = ["subject_id", "first_name", "last_name"])
df_a

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches


In [40]:
raw_data = {
    "subject_id": ["4", "5", "6", "7", "8"],
    "first_name": ["Billy", "Brian", "Bran", "Bryce", "Betty"],
    "last_name": ["Bonder", "Black", "Balwner", "Brice", "Btisan"]
}
df_b = pd.DataFrame(raw_data, columns = ["subject_id", "first_name", "last_name"])
df_b

,subject_id,first_name,last_name
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


axis의 기본값은 0으로 위아래로 붙는다.

In [42]:
df_new = pd.concat([df_a, df_b])
df_new.reset_index(drop=True)

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches
5,4,Billy,Bonder
6,5,Brian,Black
7,6,Bran,Balwner
8,7,Bryce,Brice
9,8,Betty,Btisan


**append() 함수**를 사용해도 concat() 함수와 같은 작업을 수행할 수 있다.

In [43]:
df_a.append(df_b)

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


axis = 1로 지정해보자.

In [44]:
df_new = pd.concat([df_a, df_b], axis=1)
df_new.reset_index(drop=True)

,subject_id,first_name,last_name,subject_id,first_name,last_name
0,1,Alex,Anderson,4,Billy,Bonder
1,2,Amy,Ackerman,5,Brian,Black
2,3,Allen,Ali,6,Bran,Balwner
3,4,Alice,Aoni,7,Bryce,Brice
4,5,Ayoung,Atiches,8,Betty,Btisan
